# Design Class of Tensor Markov GP

In [85]:
import torch
from dmgp.utils.sparse_design.design_class import SparseGridDesign, n_sum_k
from dmgp.utils.sparse_design.design_class import HyperbolicCrossDesign

In [86]:
# test Hyperbolic Cross Design
design_class = HyperbolicCrossDesign
deg = 3
input_bd = [0,1]

print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_lb=0, input_ub=1).points )
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_lb=0, input_ub=1).lefts)
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_lb=0, input_ub=1).rights )
print( design_class(dyadic_sort=True, return_neighbors=True)(deg=deg, input_lb=0, input_ub=1).indices_sort )

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])
tensor([  -inf,   -inf, 0.5000,   -inf, 0.2500, 0.5000, 0.7500])
tensor([   inf, 0.5000,    inf, 0.2500, 0.5000, 0.7500,    inf])
tensor([3, 1, 5, 0, 2, 4, 6])


In [87]:
# initial setting
d = 2
eta = 4
input_lb = 0
input_ub = 1
design_class = HyperbolicCrossDesign

# generate sparse grid design
sg = SparseGridDesign(d, eta, input_lb=input_lb, input_ub=input_ub, design_class=design_class).gen_sg(dyadic_sort=True, return_neighbors=True)
x_tot = sg.pts_tot
id_prt = sg.id_prt # use self.tot_pts[ id_x_prt[t_sum, prt], : ] to extract grid points in each smolyak iter
pts_prt = sg.pts_prt # use self.pts_prt[t_sum, prt] to extract a d-dimensional list, each entry is one-dim points forming the sgdesign
pts_prt_set = sg.pts_prt_set

pts_set = sg.pts_set
n_pts = sg.n_pts

design_str_prt = sg.design_str_prt


indices_prt = sg.indices_prt
indices_prt_set = sg.indices_prt_set

indices_tot = sg.indices_tot
indices_set = sg.indices_set
print(n_pts)

17


In [4]:
pts_set

tensor([[0.5000, 0.5000],
        [0.5000, 0.2500],
        [0.5000, 0.7500],
        [0.5000, 0.1250],
        [0.5000, 0.3750],
        [0.5000, 0.6250],
        [0.5000, 0.8750],
        [0.2500, 0.5000],
        [0.2500, 0.2500],
        [0.2500, 0.7500],
        [0.7500, 0.5000],
        [0.7500, 0.2500],
        [0.7500, 0.7500],
        [0.1250, 0.5000],
        [0.3750, 0.5000],
        [0.6250, 0.5000],
        [0.8750, 0.5000]])

In [5]:
design_str_prt[eta,2][0].points

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])

In [6]:
pts_prt[eta,2]

[tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750]),
 tensor([0.5000])]

In [7]:
indices_prt[eta,2]

[tensor([3, 1, 5, 0, 2, 4, 6]), tensor([3])]

In [8]:
print(indices_set)

indices_select = indices_prt_set[3,1]
print(indices_select)

tensor([[3, 3],
        [3, 1],
        [3, 5],
        [3, 0],
        [3, 2],
        [3, 4],
        [3, 6],
        [1, 3],
        [1, 1],
        [1, 5],
        [5, 3],
        [5, 1],
        [5, 5],
        [0, 3],
        [2, 3],
        [4, 3],
        [6, 3]])
tensor([[3, 3],
        [1, 3],
        [5, 3]])


In [9]:
list_full = [tuple(l) for l in indices_set.tolist()] # [n_u, d] size list of tuple
list_select = [tuple(l) for l in indices_select.tolist()] # [n_arrow, d] size list of tuple

index_dict = dict((value, idx) for idx,value in enumerate(list_full))
index_select = torch.tensor( [index_dict[x] for x in list_select] )
index_select

tensor([ 0,  7, 10])

In [10]:
i = torch.tensor([[0, 1, 2, 0],
                   [1, 1, 0, 2]])
v = torch.tensor([1, 2, 3, 4], dtype=float)
sp_test = torch.sparse_coo_tensor(i, v, [5, 5])
a = torch.randn((5,5), dtype=float)
print(a)
print(sp_test)
print(sp_test @ a)

tensor([[-2.1324, -1.6465, -1.7098,  1.8269,  1.1479],
        [ 0.7757, -0.2311, -0.0401, -0.8392,  0.4454],
        [-0.3340, -0.0328,  2.2144,  0.5889, -0.3837],
        [ 0.3290, -0.4067, -1.4507,  0.3523, -1.8992],
        [-1.3324, -1.2699, -0.3461, -0.2492, -0.6717]], dtype=torch.float64)
tensor(indices=tensor([[0, 1, 2, 0],
                       [1, 1, 0, 2]]),
       values=tensor([1., 2., 3., 4.]),
       size=(5, 5), nnz=4, dtype=torch.float64, layout=torch.sparse_coo)
tensor([[-0.5604, -0.3622,  8.8174,  1.5163, -1.0893],
        [ 1.5515, -0.4622, -0.0803, -1.6783,  0.8908],
        [-6.3971, -4.9394, -5.1293,  5.4806,  3.4437],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], dtype=torch.float64)


In [11]:
sp_id = sp_test._indices().T
print(sp_id)
print(index_select)

tensor([[0, 1],
        [1, 1],
        [2, 0],
        [0, 2]])
tensor([ 0,  7, 10])


In [12]:
index_select[sp_id]

tensor([[ 0,  7],
        [ 7,  7],
        [10,  0],
        [ 0, 10]])

In [84]:
sp_test_new = torch.sparse_coo_tensor(index_select[sp_test._indices()], sp_test._values(), [17, 17])
sp_test_new

tensor(indices=tensor([[ 0,  7, 10,  0],
                       [ 7,  7,  0, 10]]),
       values=tensor([1., 2., 3., 4.]),
       size=(17, 17), nnz=4, dtype=torch.float64, layout=torch.sparse_coo)